**Laboratory Activty**

`Instruction:` Convert the following CNN architecture diagram into a PyTorch CNN Architecture.

![image](quick_draw.png)

In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from tqdm import tqdm

# -------------------------
# Config / hyperparameters
# -------------------------
RANDOM_SEED = 42
BATCH_SIZE = 128
LR = 1e-3
NUM_EPOCHS = 3        # increase for real training
IN_CHANNELS = 1       # diagram used (1, 28, 28)
NUM_CLASSES = 10      # change to your problem's class count
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_SAVE_PATH = "diagram_cnn.pth"

# -------------------------
# Reproducibility
# -------------------------
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# -------------------------
# Model (matches diagram)
# -------------------------
class DiagramCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super().__init__()
        # Convolutional block
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.dropout = nn.Dropout(p=0.2)

        # compute flattened size for FC input with a dummy forward
        with torch.no_grad():
            d = torch.zeros(1, in_channels, 28, 28)
            d = F.relu(self.conv1(d))
            d = self.pool1(d)
            d = F.relu(self.conv2(d))
            d = F.relu(self.conv3(d))
            d = F.relu(self.conv4(d))
            d = self.pool2(d)
            flat_features = d.numel()

        # Fully connected layers (diagram: 1000, 500, final)
        self.fc1 = nn.Linear(flat_features, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, num_classes)

        self._flat_features = flat_features

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        x = self.pool2(x)
        x = self.dropout(x)

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# -------------------------
# Data (MNIST)
# -------------------------
def get_dataloaders(batch_size=BATCH_SIZE):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean/std
    ])

    train_set = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
    test_set = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    return train_loader, test_loader

# -------------------------
# Training / evaluation utilities
# -------------------------
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(dataloader, desc="Train", leave=False)
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        pbar.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Eval ", leave=False)
        for imgs, labels in pbar:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

# -------------------------
# Main training loop
# -------------------------
def main():
    print("Device:", DEVICE)
    train_loader, test_loader = get_dataloaders()

    model = DiagramCNN(in_channels=IN_CHANNELS, num_classes=NUM_CLASSES).to(DEVICE)
    print("Flatten features into FC1:", model._flat_features)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total params: {total_params:,}")

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    best_acc = 0.0
    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\n=== Epoch {epoch}/{NUM_EPOCHS} ===")
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
        val_loss, val_acc = evaluate(model, test_loader, criterion, DEVICE)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")

        # save best
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"Saved best model (val acc {best_acc:.2f}%) -> {MODEL_SAVE_PATH}")

    print("\nTraining complete. Best val accuracy: {:.2f}%".format(best_acc))

if __name__ == "__main__":
    main()

Device: cpu


100%|██████████| 9.91M/9.91M [00:10<00:00, 935kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 94.1kB/s]
100%|██████████| 1.65M/1.65M [00:04<00:00, 360kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 475kB/s]


Flatten features into FC1: 6272
Total params: 7,018,766

=== Epoch 1/3 ===


Train:   0%|          | 0/469 [00:00<?, ?it/s]d:\Anaconda\envs\myenv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Train Loss: 0.1597 | Train Acc: 94.89%
Val   Loss: 0.0531 | Val   Acc: 98.41%
Saved best model (val acc 98.41%) -> diagram_cnn.pth

=== Epoch 2/3 ===


Train Loss: 0.0447 | Train Acc: 98.70%
Val   Loss: 0.0385 | Val   Acc: 98.67%
Saved best model (val acc 98.67%) -> diagram_cnn.pth

=== Epoch 3/3 ===


Train Loss: 0.0330 | Train Acc: 98.97%
Val   Loss: 0.0313 | Val   Acc: 99.00%
Saved best model (val acc 99.00%) -> diagram_cnn.pth

Training complete. Best val accuracy: 99.00%
